<a href="https://colab.research.google.com/github/guanidine/Machine-Learning-Basics/blob/main/05.bilstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Imports
import torch
from torch import optim, nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm.contrib import tenumerate

In [2]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# Hyperparameters
input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 5

In [4]:
class BRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device)  # 2*2 x 64 x 256
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device)  # 2*2 x 64 x 256

        out, _ = self.lstm(x, (h0, c0))  # 64 x 28 x 256*2, (2*2 x 64 x 256, 2*2 x 64 x 256)
        out = self.fc(out[:, -1, :])  # 64 x 256*2 -> 64 x 10

        return out

In [5]:
# Test
model = BRNN(28, 256, 2, 10).to(device)
x = torch.randn(64, 1, 28, 28).squeeze(1).to(device)  # 64 x 28 x 28
print(model(x).shape)  # 64 x 10

torch.Size([64, 10])


In [6]:
# Load Data
train_dateset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dateset, batch_size=batch_size, shuffle=True)
test_dateset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dateset, batch_size=batch_size, shuffle=True)

In [7]:
# Initialize network
model = BRNN(input_size, hidden_size, num_layers, num_classes).to(device)

In [8]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
# Train network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in tenumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device)  # 64 x 1 x 28 x 28
        targets = targets.to(device)

        data = data.squeeze(1)

        # Forward
        scores = model(data)
        loss = criterion(scores, targets)

        # Backward
        optimizer.zero_grad()
        loss.backward()

        # Gradient descent or adam step
        optimizer.step()

  0%|          | 0/938 [00:00<?, ?it/s]

  0%|          | 0/938 [00:00<?, ?it/s]

  0%|          | 0/938 [00:00<?, ?it/s]

  0%|          | 0/938 [00:00<?, ?it/s]

  0%|          | 0/938 [00:00<?, ?it/s]

In [10]:
# Check accuracy on training & test
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            scores = model(x.squeeze(1))
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(f"Got {num_correct} / {num_samples} with accuracy {float(num_correct) / float(num_samples) * 100:.2f}")

    model.train()
    return num_correct / num_samples


check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 59316 / 60000 with accuracy 98.86
Checking accuracy on test data
Got 9856 / 10000 with accuracy 98.56


tensor(0.9856, device='cuda:0')